_HDS5210 Programming for Health Data Science_

# Week 12 - Introduction to Databases

This week, we're talking about different types of databases and database concepts.


1 - All Rows
---

Use SQL to retrieve a list of all of the records from plans_by_state and then count the results.  Do not aggregate or filter the results from the database.  Retreive all the records AND THEN count the results.

In [2]:
import pymysql
import pandas as pd

connection = pymysql.connect(
    host='localhost',
    user='slucor',
    password='s1uc0r',
    db='hds5210')

cursor = connection.cursor()
# Read a single record
sql = "SELECT * FROM plans_by_state"
sql = "SELECT COUNT(*) FROM plans_by_state"
sql = "SELECT COUNT(state_name) FROM plans_by_state"
cursor.execute(sql)
results = cursor.fetchall()
connection.close()

# Count just by getting the length of the results list
len(results)

10213

2 - Only Arizona plans
---

Which of the following queries give us a result with only plans from Arizona?

A. `SELECT * FROM orgs_by_state WHERE state_name = 'Arizona'`

B. `SELECT * WHERE state_name = 'Arizona' * FROM plans_by_state`

C. `WHERE state_name = 'Arizona' SELECT * FROM plans_by_state`

**D. `SELECT * FROM plans_by_state WHERE state_name='Arizona'`**

**Answer:**  D

3 - How Many Plans per State?
---

Write a query that aggregates the plan information to return only one row per state and a count of plans for each state.

In [16]:
import pymysql
import pandas as pd

connection = pymysql.connect(
    host='localhost',
    user='slucor',
    password='s1uc0r',
    db='hds5210',    cursorclass=pymysql.cursors.DictCursor)

cursor = connection.cursor()
# Read a single record
sql = "SELECT state_name, count(*) FROM plans_by_state GROUP BY state_name"
cursor.execute(sql)
results = cursor.fetchall()
connection.close()

results

[{'count(*)': 1, 'state_name': ''},
 {'count(*)': 174, 'state_name': 'Alabama'},
 {'count(*)': 74, 'state_name': 'Alaska'},
 {'count(*)': 198, 'state_name': 'Arizona'},
 {'count(*)': 296, 'state_name': 'Arkansas'},
 {'count(*)': 173, 'state_name': 'California - Northern & Central'},
 {'count(*)': 173, 'state_name': 'California - Southern'},
 {'count(*)': 196, 'state_name': 'Colorado'},
 {'count(*)': 92, 'state_name': 'Connecticut'},
 {'count(*)': 146, 'state_name': 'Delaware'},
 {'count(*)': 57, 'state_name': 'District of Columbia'},
 {'count(*)': 156, 'state_name': 'Florida - Northern'},
 {'count(*)': 156, 'state_name': 'Florida - Southern'},
 {'count(*)': 161, 'state_name': 'Georgia'},
 {'count(*)': 7, 'state_name': 'Guam'},
 {'count(*)': 76, 'state_name': 'Hawaii'},
 {'count(*)': 232, 'state_name': 'Idaho'},
 {'count(*)': 256, 'state_name': 'Illinois'},
 {'count(*)': 256, 'state_name': 'Indiana'},
 {'count(*)': 236, 'state_name': 'Iowa'},
 {'count(*)': 286, 'state_name': 'Kansas'},


4 - How Many Plans and Orgs per State?
---

Aggregate the Orgs by State and Plans by State and count the total number of orgs and plans.  Then combine those results together.  The result that we're looking for will have one row per state and include columns for state_name, plan_count, and org_count.  You can do this either using entirely SQL or you can do it by retreiveing a part of the results from the database and then doing the rest of the work in Python.


In [11]:
import pymysql
import pandas as pd

connection = pymysql.connect(
    host='localhost',
    user='slucor',
    password='s1uc0r',
    db='hds5210',
    cursorclass=pymysql.cursors.DictCursor)


cursor = connection.cursor()

# Get orgs by state
sql = "SELECT state_name, count(1) AS plan_count FROM plans_by_state GROUP BY state_name"
cursor.execute(sql)
plans = cursor.fetchall()

sql = "SELECT state_name, count(1) AS org_count FROM orgs_by_state GROUP BY state_name"
cursor.execute(sql)
orgs = cursor.fetchall()

connection.close()

In [14]:
merged = []
for p in plans:
    for o in orgs:
        if o['state_name'] == p['state_name']:
            merged.append([p['state_name'], p['plan_count'], o['org_count']])

# Bonus question... what's inefficient about my approach above?

In [15]:
merged

[['Alabama', 174, 41],
 ['Alaska', 74, 16],
 ['Arizona', 198, 52],
 ['Arkansas', 296, 52],
 ['California - Northern & Central', 173, 29],
 ['California - Southern', 173, 29],
 ['Colorado', 196, 45],
 ['Connecticut', 92, 16],
 ['Delaware', 146, 27],
 ['District of Columbia', 57, 11],
 ['Florida - Northern', 156, 31],
 ['Florida - Southern', 156, 31],
 ['Georgia', 161, 36],
 ['Guam', 7, 1],
 ['Hawaii', 76, 13],
 ['Idaho', 232, 47],
 ['Illinois', 256, 49],
 ['Indiana', 256, 60],
 ['Iowa', 236, 51],
 ['Kansas', 286, 61],
 ['Kentucky', 264, 54],
 ['Louisiana', 186, 39],
 ['Maine', 103, 21],
 ['Maryland', 162, 31],
 ['Massachusetts', 20, 10],
 ['Michigan', 152, 79],
 ['Minnesota', 88, 25],
 ['Mississippi', 125, 61],
 ['Missouri', 265, 56],
 ['Montana', 220, 40],
 ['Nebraska', 201, 47],
 ['Nevada', 160, 44],
 ['New Hampshire', 109, 24],
 ['New Jersey', 133, 27],
 ['New Mexico', 247, 48],
 ['New York - New York City Vicinity', 83, 18],
 ['New York - Upstate', 83, 18],
 ['North Carolina', 275, 

In [17]:
import pymysql
import pandas as pd

connection = pymysql.connect(
    host='localhost',
    user='slucor',
    password='s1uc0r',
    db='hds5210',
    cursorclass=pymysql.cursors.DictCursor)

Big_Count=pd.read_sql("""
Select 
  a.*, 
  b.org_count 
from (
      select 
        case when state_name like '%-%' then SUBSTRING_INDEX(state_name, '-', 1) else state_name end as state_name2,
        count(*) as plan_count 
      from plans_by_state 
      group by state_name2 
  ) a inner join (
      select 
        case when state_name like '%-%' then SUBSTRING_INDEX(state_name, '-', 1) else state_name end as state_name2,
        count(*) as org_count 
      from orgs_by_state 
      group by state_name2
  ) b on a.state_name2=b.state_name2
""",connection)
Big_Count

,state_name2,plan_count,org_count
0,Alabama,174,41
1,Alaska,74,16
2,Arizona,198,52
3,Arkansas,296,52
4,California,346,58
5,Colorado,196,45
6,Connecticut,92,16
7,Delaware,146,27
8,District of Columbia,57,11
9,Florida,312,62


In [ ]:
""
SELECT
   pbs.state_name
   , COUNT(pbs.name) AS plan_count
   , ( SELECT 
         COUNT(obs.name) AS org_count 
       FROM orgs_by_state AS obs 
       WHERE obs.state_name = pbs.state_name 
       GROUP BY state_name) AS org_count
FROM plans_by_state AS pbs
GROUP BY state_name
"""

In [ ]:
"""
SELECT 
  p.state_name, 
  p.plan_count,
  o.org_count
FROM d
  ( SELECT state_name, COUNT(*) AS plan_count FROM plans_by_state GROUP BY state_name) AS p INNER JOIN 
  ( SELECT state_name, COUNT(*) AS org_count FROM orgs_by_state GROUP BY state_name) AS o ON p.state_name=o.state_name
GROUP BY p.state_name
"""